In [5]:
import geopandas as gpd
import matplotlib.pyplot as plt
import rasterio
from rasterio.plot import show
from shapely.geometry import Point
import pandas as pd
import numpy as np

In [6]:
# !pip install rasterio

In [8]:
stations_df = pd.read_csv('data/stations.csv')
glorich_df = pd.read_csv('data/final_imputed_glorich.csv')
landsat_df = pd.read_csv('data/landsat_features_training.csv')

In [11]:
def df_to_gdf(df):
    df["geometry"] = [Point(xy) for xy in zip(df["Longitude"], df["Latitude"])]
    return gpd.GeoDataFrame(df, geometry="geometry", crs="EPSG:4326")

In [17]:
landsat_to_station.head()

,Latitude_left,Longitude_left,Sample Date,nir,green,swir16,swir22,NDMI,MNDWI,geometry,...,SpecCond25C,pH,Alkalinity_reliability,Cl_reliability,DIP_reliability,SO4_reliability,SpecCond25C_reliability,pH_reliability,dist_m,dist_km
0,-28.760833,17.730278,02-01-2011,11190.0,11426.0,7687.5,7645.0,0.185538,0.195595,POINT (17.73028 -28.76083),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.793981e+06,3793.981405
1,-26.861111,28.884722,03-01-2011,17658.5,9550.0,13746.5,10574.0,0.124566,-0.180134,POINT (28.88472 -26.86111),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.793978e+06,3793.977668
2,-26.450000,28.085833,03-01-2011,15210.0,10720.0,17974.0,14201.0,-0.083293,-0.252805,POINT (28.08583 -26.45),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.793978e+06,3793.978414
3,-27.671111,27.236944,03-01-2011,14887.0,10943.0,13522.0,11403.0,0.048048,-0.105416,POINT (27.23694 -27.67111),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.793978e+06,3793.977757
4,-27.356667,27.286389,03-01-2011,16828.5,9502.5,12665.5,9643.0,0.141147,-0.142683,POINT (27.28639 -27.35667),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.793978e+06,3793.978008


In [12]:
glorich_gdf = df_to_gdf(glorich_df)
landsat_gdf = df_to_gdf(landsat_df)

In [22]:
stations_m = glorich_gdf.to_crs(epsg=3857)
landsat_m  = landsat_gdf.to_crs(epsg=3857)

landsat_to_station = gpd.sjoin_nearest(
    landsat_gdf,                 # LEFT: each landsat point
    stations_m,  # RIGHT: station candidates
    how="left",
    distance_col="dist_m"
)

landsat_to_station["dist_km"] = landsat_to_station["dist_m"] / 1000
landsat_to_station = landsat_to_station.rename(columns= {'Latitude_left': 'Latitude', 'Longitude_left': 'Longitude', 'Latitude_right': 'Latitude_stations', 'Longitude_right': 'Longitude_stations'})

landsat_to_station.head()

C:\Users\knich\AppData\Local\Temp\ipykernel_5228\2980896796.py:4: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: EPSG:4326
Right CRS: EPSG:3857

  landsat_to_station = gpd.sjoin_nearest(
C:\Users\knich\miniconda3\Lib\site-packages\geopandas\array.py:407: UserWarning: Geometry is in a geographic CRS. Results from 'sjoin_nearest' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  warnings.warn(


,Latitude,Longitude,Sample Date,nir,green,swir16,swir22,NDMI,MNDWI,geometry,...,SpecCond25C,pH,Alkalinity_reliability,Cl_reliability,DIP_reliability,SO4_reliability,SpecCond25C_reliability,pH_reliability,dist_m,dist_km
0,-28.760833,17.730278,02-01-2011,11190.0,11426.0,7687.5,7645.0,0.185538,0.195595,POINT (17.73028 -28.76083),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.793981e+06,3793.981405
1,-26.861111,28.884722,03-01-2011,17658.5,9550.0,13746.5,10574.0,0.124566,-0.180134,POINT (28.88472 -26.86111),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.793978e+06,3793.977668
2,-26.450000,28.085833,03-01-2011,15210.0,10720.0,17974.0,14201.0,-0.083293,-0.252805,POINT (28.08583 -26.45),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.793978e+06,3793.978414
3,-27.671111,27.236944,03-01-2011,14887.0,10943.0,13522.0,11403.0,0.048048,-0.105416,POINT (27.23694 -27.67111),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.793978e+06,3793.977757
4,-27.356667,27.286389,03-01-2011,16828.5,9502.5,12665.5,9643.0,0.141147,-0.142683,POINT (27.28639 -27.35667),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.793978e+06,3793.978008
